In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as klayers
import numpy as np

np.set_printoptions(precision=4, suppress=True)

import gradnet
from gradnet import layers as glayers

In [2]:
np.random.seed(123)

mb = 1
x = np.random.random((mb, 4, 4, 1))
y_ = np.array([[0.0]]*mb)



In [3]:
def kmodel():
    inp = klayers.Input((4, 4, 1), name="input")
    conv = klayers.Conv2D(3, 3, activation="tanh", name="conv")(inp)
    f = klayers.Flatten()(conv)
    conc = klayers.Concatenate()([f,f])
    out = klayers.Dense(1, name="dense")(conc)
    model = keras.Model(inp, out)
    return model, conv

def gmodel():
    inp = gradnet.Input((4, 4, 1), name="input")
    conv = glayers.Conv2D(3, 3, 3, activation="tanh", name="conv")([inp])
    f = glayers.Flatten()(conv)
    print("flatten shape:", f.Shape)
    conc = glayers.Concatenate()([f,f])
    print("concatenate shape:", conc.Shape)
    out = glayers.Dense(1, name="dense")(conc)
    print("out shape:", out.Shape)
    model = gradnet.Model([inp], [out])
    model.add_loss(gradnet.losses.get_loss("mse")(out))
    return model, conv


In [4]:
km, kconv = kmodel()
km.summary()

kweights = {}

for l in km.layers:
    wlst = l.get_weights()
    if wlst:
        print(l.name)
        for w in wlst:
            print("   ", w)
        kweights[l.name] = wlst

gm, gconv = gmodel()
for l in gm.layers:
    print(type(l), l)
    name = l.Name
    if name:
        w = kweights[name]
        #print("keras weight for", name,":", w)
        l.set_weights(w)
        print(name, "weights:", l.get_weights())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 4, 4, 1)]    0                                            
__________________________________________________________________________________________________
conv (Conv2D)                   (None, 2, 2, 3)      30          input[0][0]                      
__________________________________________________________________________________________________
flatten (Flatten)               (None, 12)           0           conv[0][0]                       
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 24)           0           flatten[0][0]                    
                                                                 flatten[0][0]                

In [7]:
# compare models
print("x:\n", x)
ky = km(x).numpy()
print("Keras y:\n", ky)
gy = gm.call([x])
print("Gradnet y:\n", gy)

print


x:
 [[[[0.6965]
   [0.2861]
   [0.2269]
   [0.5513]]

  [[0.7195]
   [0.4231]
   [0.9808]
   [0.6848]]

  [[0.4809]
   [0.3921]
   [0.3432]
   [0.729 ]]

  [[0.4386]
   [0.0597]
   [0.398 ]
   [0.738 ]]]]
Keras y:
 [[-0.1677]]
Gradnet y:
 [array([[-0.1677]])]


<function print>

In [8]:
def train_keras(x, y_, model):
    with tf.GradientTape() as tape:
        loss = keras.losses.MSE
        losses = []
        for xi, yi_ in zip(x, y_):
            xi = xi[None,:]
            yi_ = yi_[None,:]
            xi = tf.convert_to_tensor(xi)
            yi = model(xi)
            #print("xi=", xi, "  yi_=", yi_, "  yi=", yi)
            losses.append(loss(yi_, yi))
        total_loss = sum(losses)
        grads = tape.gradient(total_loss, model.trainable_variables)
    return [g.numpy() for g in grads]


In [9]:
kgrads = train_keras(x, y_, km)
print("Keras gradients:")
for g in kgrads:
    print(g)

Keras gradients:
[[[[-0.0403  0.0233 -0.1121]]

  [[-0.0089 -0.002  -0.0134]]

  [[-0.0024 -0.0122 -0.0358]]]


 [[[-0.0402  0.0486 -0.0945]]

  [[-0.0124  0.0708 -0.0193]]

  [[-0.0519  0.089  -0.0951]]]


 [[[-0.0269  0.0362 -0.0736]]

  [[-0.0189  0.0433 -0.0249]]

  [[-0.0075  0.0416 -0.0042]]]]
[-0.0466  0.07   -0.111 ]
[[ 0.1649]
 [-0.1408]
 [ 0.0534]
 [ 0.1156]
 [ 0.0337]
 [ 0.1598]
 [ 0.0671]
 [-0.0114]
 [ 0.1205]
 [ 0.1473]
 [-0.1048]
 [ 0.1283]
 [ 0.1649]
 [-0.1408]
 [ 0.0534]
 [ 0.1156]
 [ 0.0337]
 [ 0.1598]
 [ 0.0671]
 [-0.0114]
 [ 0.1205]
 [ 0.1473]
 [-0.1048]
 [ 0.1283]]
[-0.3353]


In [10]:
def train_gradnet(x, y_, model):
    model.reset_losses()
    for xi, yi_ in zip(x, y_):
        xi = xi[None,:]
        yi_ = yi_[None,:]
        yi = model.call([xi])
        #print("call: xi:", xi, "  -> yi:", yi, "   yi_:", yi_)
        model.backprop(yi_)
    return model.layer_gradients() 

def train_gradnet(x, y_, model):
    model.reset_losses()
    model.call([x])
    model.backprop(y_)
    for l in model.layers:
        if l.PGradSum is not None:
            print(l.Name, l.PGradSum)
    return model.layer_gradients() 


In [11]:
ggrads = train_gradnet(x, y_, gm)
print("Gradnet gradients:")
for g in ggrads:
    print(g)

concatente: ygrads: [[-0.0005 -0.0115 -0.0284  0.0646  0.1489 -0.0035  0.0766 -0.1548 -0.13
  -0.0511  0.0722  0.0865 -0.0897  0.1013 -0.119  -0.0485 -0.0796  0.0713
  -0.0765  0.0967  0.0576  0.0605 -0.0883 -0.0357]]  --->  [array([[-0.0005, -0.0115, -0.0284,  0.0646,  0.1489, -0.0035,  0.0766,
        -0.1548, -0.13  , -0.0511,  0.0722,  0.0865]]), array([[-0.0897,  0.1013, -0.119 , -0.0485, -0.0796,  0.0713, -0.0765,
         0.0967,  0.0576,  0.0605, -0.0883, -0.0357]])]
dense [array([[ 0.1649],
       [-0.1408],
       [ 0.0534],
       [ 0.1156],
       [ 0.0337],
       [ 0.1598],
       [ 0.0671],
       [-0.0114],
       [ 0.1205],
       [ 0.1473],
       [-0.1048],
       [ 0.1283],
       [ 0.1649],
       [-0.1408],
       [ 0.0534],
       [ 0.1156],
       [ 0.0337],
       [ 0.1598],
       [ 0.0671],
       [-0.0114],
       [ 0.1205],
       [ 0.1473],
       [-0.1048],
       [ 0.1283]]), array([-0.3353])]
conv [array([[[[-0.0403,  0.0233, -0.1121]],

        [[-0.00